## Postprocessing randomized measurement on IBM quantum computers (

This tutorial illustrates how to acquire and save randomized measurements on a quantum computer using Qiskit (RandomizedMeasurementsQiskit.ipynb). This data can be then postprocessed using our julia library RandomMeas (RandomizedMeasurementsQiskitPostprocessing.ipynb)

In [1]:
using RandomMeas
using NPZ

[ Info: Precompiling RandomMeas [a9f96d1f-ed6a-4404-822f-5927c777f274] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


### Loading the data and retrieving the parameters of the experiment

In [2]:
job_id = "1f502b5b-496c-4b68-b3dd-e35780bbdf8a"
basis_transformation = npzread("data/"*job_id*"_u.npy")
measurements_cal = 2 .- npzread("data/"*job_id*"_data.npy")[1,:,:,:]; #Data obtained from the circuit qc_meas used for calibrating robust shadows
measurements = 2 .- npzread("data/"*job_id*"_data.npy")[2,:,:,:]; #Data obtained from the preparation of the cluster state
NU,NM,N = size(measurements)
@show NU,NM,N;

(NU, NM, N) = (2048, 1024, 4)


### Converting the data into RandomMeas.jl format

We use a MeasurementGroup constructor using the bitstrings and the basis transformations as input arguments

In [3]:
ξ = siteinds("Qubit",N)
group_cal = MeasurementGroup(measurements_cal,basis_transformation,ξ);
group = MeasurementGroup(measurements,basis_transformation,ξ);

### Extraction of the calibration parameters $G$ for building robust shadows

For purely readout error models, we have $G\approx 1-p$ where $p$ is the readout error 

In [4]:
states = ["Dn" for n in 1:N]
ψ0  = MPS(ComplexF64,ξ,states);
G_e = get_calibration_vector(ψ0,group_cal)
print("Calibrated RM parameters: ", G_e)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:35


Calibrated RM parameters: [0.9855832110091116, 0.9769867562233032, 0.9810202636856729, 0.9795555286370551]

### Extracting the purity

We extract the purity of the first two qubits, and the purity of the four qubits. As expected for a cluster state, we have $p(12)\approx 0.5$, and $p(1234)\approx 1$.  

In [5]:
ρ = get_dense_shadows(group,number_of_ru_batches=10)
ρ_sub = partial_trace(ρ,collect(1:2))
ρ_mitigated = get_dense_shadows(group,number_of_ru_batches=10;G=G_e)
ρ_mitigated_sub = partial_trace(ρ_mitigated,collect(1:2))
println("SubSystem Purity (unmitigated) ", get_trace_moment(ρ_sub,2))
println("SubSystem Purity (mitigated) ", get_trace_moment(ρ_mitigated_sub,2))
println("Total Purity (unmitigatd) ", get_trace_moment(ρ,2))
println("Total Purity (mitigated) ", get_trace_moment(ρ_mitigated,2))

SubSystem Purity (unmitigated) 0.46876954314660957
SubSystem Purity (mitigated) 0.50487483432288
Total Purity (unmitigatd) 0.8125488264803807
Total Purity (mitigated) 1.022254735212368
